In [37]:
#Import Libraries
import pandas as pd
import requests
import io
import numpy as np
import json
import lime
import lime.lime_tabular

In [20]:
# # Downloading the csv file from the GitHub account
# url = "http://localhost:8888/edit/Documents/GitHub/mirahealth/datasets/mira_data_one_hot.csv" # Make sure the url is the raw version of the file on GitHub
# download = requests.get(url).content
# Reading the downloaded content and turning it into a pandas dataframe
df = pd.read_csv('mira_data_one_hot.csv')
# Printing out the first 5 rows of the dataframe
df.drop('Unnamed: 0', inplace=True, axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2262 entries, 0 to 2261
Data columns (total 31 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   obgyn_AvgPatientTime         2262 non-null   int64  
 1   obgyn_NumOfPatients          2262 non-null   int64  
 2   obgyn_QualityRisk            2262 non-null   int64  
 3   obgyn_TotalHours             2262 non-null   int64  
 4   obgyn_BurnoutRisk            2262 non-null   int64  
 5   Age                          2262 non-null   int64  
 6   SystolicBP                   2262 non-null   int64  
 7   DiastolicBP                  2262 non-null   int64  
 8   BS                           2262 non-null   float64
 9   BodyTemp                     2262 non-null   float64
 10  HeartRate                    2262 non-null   int64  
 11  HDP                          2262 non-null   int64  
 12  obgyn_Location_Metropolitan  2262 non-null   int64  
 13  obgyn_Location_Mic

In [3]:
# shuffle the DataFrame rows
df = df.sample(frac = 1, ignore_index=True)
df

,obgyn_AvgPatientTime,obgyn_NumOfPatients,obgyn_QualityRisk,obgyn_TotalHours,obgyn_BurnoutRisk,Age,SystolicBP,DiastolicBP,BS,BodyTemp,...,Payer_Public,Payer_Uninsured,Location_Metropolitan,Location_Micropolitan,Location_Rural,Income_Q1,Income_Q2,Income_Q3,Income_Q4,obgyn_sentiment
0,11,20,1,220,0,37,120,80,7.8,98.0,...,0,0,1,0,0,0,1,0,0,1
1,18,23,0,414,1,36,90,60,7.5,98.0,...,1,0,1,0,0,0,0,0,1,0
2,18,29,0,522,1,26,130,100,16.0,98.0,...,0,1,1,0,0,0,1,0,0,0
3,15,23,0,345,0,25,120,90,7.8,98.0,...,0,0,0,1,0,0,0,0,1,1
4,17,16,0,272,0,16,115,60,7.8,101.0,...,0,0,0,0,1,0,0,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2257,11,20,1,220,0,28,130,70,12.0,98.0,...,1,0,0,1,0,0,0,0,1,1
2258,11,20,1,220,0,34,85,60,6.9,98.0,...,1,0,1,0,0,0,0,0,1,1
2259,19,30,0,570,1,40,140,80,6.7,98.0,...,0,1,1,0,0,0,0,0,1,1
2260,20,15,0,300,0,41,130,70,12.0,98.0,...,1,0,0,0,1,0,0,0,1,1


In [4]:
#split dataset in features and target variable
feature_cols = df.columns.values.tolist()
feature_cols.remove('HDP')
X = df[feature_cols] # Features
y = df.HDP # Target variable

In [5]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)

In [6]:
# import the class
from sklearn.linear_model import LogisticRegression
# instantiate the model (using the default parameters)
logreg = LogisticRegression()
# fit the model with data
logreg.fit(X_train, y_train)


/Users/brianp/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [7]:
#Score the Accuracy of the model
from sklearn.metrics import accuracy_score
y_pred = logreg.predict(X_test)
score =accuracy_score(y_test,y_pred)
print('Test Accuracy Score: ', score)

Test Accuracy Score:  0.9098939929328622


In [8]:
#Printing Test Set as a Reference
y_test.values.tolist()
X_test["HDP"]=y_test

In [9]:
X_test.to_csv('test_set.csv')

In [10]:
#Save the persistent model
from joblib import dump, load
dump(logreg, '/Users/brianp/Documents/GitHub/mirahealth/mirahealth_api/mirahealth_v1_0.joblib') 

['/Users/brianp/Documents/GitHub/mirahealth/mirahealth_api/mirahealth_v1_0.joblib']

In [13]:
test_json = (X_test.iloc[111]).to_json()
test_json
# #convert dictionary to a json object
# json_object = json.dumps(test_json)
# json_object

'{"obgyn_AvgPatientTime":14.0,"obgyn_NumOfPatients":28.0,"obgyn_QualityRisk":0.0,"obgyn_TotalHours":392.0,"obgyn_BurnoutRisk":1.0,"Age":17.0,"SystolicBP":140.0,"DiastolicBP":95.0,"BS":17.0,"BodyTemp":98.0,"HeartRate":60.0,"obgyn_Location_Metropolitan":1.0,"obgyn_Location_Micropolitan":0.0,"obgyn_Location_Rural":0.0,"Ethnicity_Asian_Pacific":0.0,"Ethnicity_Black":0.0,"Ethnicity_Hispanic":0.0,"Ethnicity_Native":0.0,"Ethnicity_White":1.0,"Payer_Private":1.0,"Payer_Public":0.0,"Payer_Uninsured":0.0,"Location_Metropolitan":0.0,"Location_Micropolitan":1.0,"Location_Rural":0.0,"Income_Q1":0.0,"Income_Q2":0.0,"Income_Q3":0.0,"Income_Q4":1.0,"obgyn_sentiment":0.0,"HDP":1.0}'

In [14]:
#Convert json object to dataframe
input = pd.read_json(test_json, orient="index").transpose()
input = input.drop('HDP', axis=1)
input

,obgyn_AvgPatientTime,obgyn_NumOfPatients,obgyn_QualityRisk,obgyn_TotalHours,obgyn_BurnoutRisk,Age,SystolicBP,DiastolicBP,BS,BodyTemp,...,Payer_Public,Payer_Uninsured,Location_Metropolitan,Location_Micropolitan,Location_Rural,Income_Q1,Income_Q2,Income_Q3,Income_Q4,obgyn_sentiment
0,14,28,0,392,1,17,140,95,17,98,...,0,0,0,1,0,0,0,0,1,0


In [36]:
input

,obgyn_AvgPatientTime,obgyn_NumOfPatients,obgyn_QualityRisk,obgyn_TotalHours,obgyn_BurnoutRisk,Age,SystolicBP,DiastolicBP,BS,BodyTemp,...,Payer_Public,Payer_Uninsured,Location_Metropolitan,Location_Micropolitan,Location_Rural,Income_Q1,Income_Q2,Income_Q3,Income_Q4,obgyn_sentiment
0,14,28,0,392,1,17,140,95,17,98,...,0,0,0,1,0,0,0,0,1,0


In [15]:
y_output = logreg.predict(input)
print (y_output)

[1]


In [16]:
score = logreg.predict_proba(input)[:,1]
print (score)

[0.97786584]


In [ ]:
#lime
explainer = lime_tabular.LimeTabularExplainer(training_data = np.array(X_train),
                                             mode = "regression",
                                             feature_names = X_train.columns,
                                             categorical_features = ['HDP'])